In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [2]:
from numpy.linalg import norm
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ENTER YOUR CODE HERE   
    y = np.zeros(len(vec))
    y[0] = norm(vec)  
    u = ((vec-y)/norm(vec-y)).reshape(len(vec),1)
    h = np.eye(len(vec)) - 2*np.dot(u,u.reshape(1,len(vec)))
    return y, h

In [3]:
# Test (marked)

v = np.array([1, 2, 3])
v1, h = householder(v)

# print(v1)
# print(h)
# print(np.dot(h,h.T))

assert_allclose(np.dot(h, v1), v)
#assert_allclose(np.dot(h, v), v1)
assert_allclose(np.dot(h, v), v1, atol=1e-12)

In [4]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [5]:
from numpy.linalg import inv
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # ENTER YOUR CODE HERE
    
    q = np.eye(m)
    r = a1
    for k in range(n):
        cur_col = r[:,k][k:]
        y, h_cur = householder(cur_col)
        h_ = np.eye(m)
        h_[k:,k:] = h_cur
        q = np.dot(h_,q)
        r = np.dot(h_,r)
    return inv(q), r

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [7]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)
print(q)
print(r)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.          0.          0.5496046 ]
 [ 0.         -0.         -0.        ]
 [ 0.          0.          0.        ]]


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)
print(qq)
print(rr)

assert_allclose(np.dot(qq, rr), a)

[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here (Russian or English, up to you)*

Некоторые элементы соответствующих матриц отличаются только знаками, которые при умножении матриц q*r и qq*rr дают одинаковый результат в виде матрицы a. Различие в знаках можно объяснить выбором вектора нормали к гиперплоскости на каждом шаге

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [9]:
def qr_without_H(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    U = []
    for k in range(n):
        a_ = a1[k:,k:]
        vec = a_[:,0]
        y = np.zeros(len(vec))
        y[0] = norm(vec)
        u = ((vec - y)/norm(vec-y)).reshape(-1,1)
        a1[k:,k:] = a1[k:,k:] - 2*np.dot(u, np.dot(u.T,a1[k:,k:]))
        u = np.insert(u, 0, np.zeros(m - u.shape[0])).reshape(-1,1)
        U.append(u)
    return a1, U

In [10]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5,3))
rr, uu = qr_without_H(a)

In [11]:
np.allclose(r, rr)

True

In [14]:
def check(a, U):
    a1 = a
    for u in U:
        a1 = a1 - 2*np.dot(u, np.dot(u.T, a1))
    return a1

In [15]:
np.allclose(check(a, uu), rr)

True